# Logistic Regression

[wikipedia](https://en.wikipedia.org/wiki/Logistic_regression)

En principio no hay gran diferencia en la frontera de decisión final entre el ajuste lineal directo de las etiquetas +1 -1 con la pseudoinversa y la técnica más correcta de _iterative reweighted least squares_.

La diferencia está en que la regresión logística estima la probabilidad de las etiquetas. Se asume un experimento aleatorio, no un "ruido de medida".

Recordemos que MSE lineal va más o menos bien a pesar de que penaliza etiquetas muy bien clasificadas dentro de la frontera de decisión. Por el contrario, ajustar bien la logistic regression es un proceso no lineal, y realmente obtiene las probabilidades de pertenencia, lo cual puede ser importante a la hora de predecir. No sólo te da la frontera, sino lo abrupta que es la transición de probabilidades.

La idea genial es estimar linealmente (o con un modelo lineal generalizado con features) los _logodds_.

En este experimento vamos a generar un dataset artificial obteniendo $P(c=1) = \sigma(w^Tx)$, donde $\sigma(·)$ es la típica nolinealidad suave. En la zona de probabilidad intermedia las clases se entremezclan de forma aleatoria.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

inv = np.linalg.inv
pinv = np.linalg.pinv

def sigma(x):
    return 1/(1+np.exp(-x))

# dibuja una recta "infinita" dadas sus coordenadas homogéneas
def shline(l,xmin=-2000,xmax=2000, **kwargs):
    a,b,c = l / np.linalg.norm(l)
    if abs(b) < 1e-6:
        x = -c/a
        r = np.array([[x,xmin],[x,xmax]])
    else:
        y0 = (-a*xmin - c) / b
        y1 = (-a*xmax - c) / b
        r = np.array([[xmin,y0],[xmax,y1]])
    plt.plot(*r.T, **kwargs)

In [ ]:
w = [2, -1, 1]
#w = [1, -0.5, 1]
#w = [20, -10, 1]

data = np.random.randn(1000,2)*2


X = np.hstack([data, np.ones([len(data),1])])
X.shape
logodds = X@w

probs = sigma(logodds)

y = np.array([np.random.choice([1,0], p=[p,1-p]) for p in probs])

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(*data.T,10,y,cmap='coolwarm',alpha=1);
plt.axis('equal')
ax = plt.axis()
shline(w,color='black')
plt.axis(ax);
plt.show()

plt.figure(figsize=(6,6))
plt.scatter(*data[y==0].T,10,'blue');
shline(w,color='black')
plt.axis(ax);
plt.show()

plt.figure(figsize=(6,6))
plt.scatter(*data[y==1].T,10,'red');
shline(w,color='black')
plt.axis(ax);
plt.show()

plt.figure(figsize=(6,6))
plt.scatter(*data.T,10,logodds,cmap='coolwarm',alpha=1,vmin=-3,vmax=3);
shline(w,color='black')
plt.axis(ax)
plt.show()

Vamos a intentar estimar los parámetros de modelo mediante el algoritmo iterativo:

In [ ]:
wlr = np.random.rand(3)/10

for _ in range(15):
    mu =  sigma(X@wlr)
    S = np.diag( mu*(1-mu) )
    wlr = inv(X.T @ S @ X)@X.T@(S@X@wlr + y - mu)
    print(wlr)

La solución obtenida se parece mucho a la verdadera:

In [ ]:
wye = pinv(X)@(2*y-1)
print(wye)
wye/abs(wye[-1])

In [ ]:
wmse = pinv(X)@logodds
wmse

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(*data.T,10, X@wlr ,cmap='coolwarm',alpha=1,vmin=-3,vmax=3);
ax = plt.axis()
shline(w)
#shline(wmse,color='gray')
shline(wlr,color='green')
shline(wye,color='blue')
shline(wlr+[0,0,3],color='green',lw=0.5)
shline(wlr+[0,0,-3],color='green',lw=0.5)
shline(np.array(w)+[0,0,3],color='gray')
shline(np.array(w)+[0,0,-3],color='gray')

plt.axis(ax);

La estimación completa del modelo logístico nos devuelve también el ancho de la franja donde hay incertidumbre en la clasificación. En este ejemplo hemos puesto las fronteras con un logodds de $\pm 3$ que corresponde con una probabilidad de

In [ ]:
sigma(3)